# Query analysis

In [17]:
from langserve import RemoteRunnable

In [18]:
from typing import Optional

from langchain_core.pydantic_v1 import BaseModel, Field


class Search(BaseModel):
    """Search over a database of tutorial videos about a software library."""  # noqa

    query: str = Field(
        ...,
        description="Similarity search query applied to video transcripts.",  # noqa
    )
    publish_year: Optional[int] = Field(
        None, description="Year video was published"
    )

In [19]:
runnable = RemoteRunnable("http://localhost:8000/query_analysis/")

In [20]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage("RAG agent tutorial from 2023")]
response = runnable.invoke(
    {"messages": messages, "schema": Search.schema()}
)
response

{'data': [{'query': 'RAG agent tutorial', 'publish_year': 2023}]}

Add instructions:

In [23]:
instructions = (
    "Always expand acronym RAG to Retrieval Augmented Generation. "
    "NEVER INCLUDE RAG IN THE SEARCH"
)

response = runnable.invoke(
    {
        "messages": messages,
        "schema": Search.schema(),
        "instructions": instructions,
    }
)
response

{'data': [{'query': 'RAG agent tutorial', 'publish_year': 2023}]}

Add few-shot examples:

In [24]:
examples = [
    {
        "messages": [HumanMessage("RAG from scratch series")],
        "output": [ 
            {"search": "Retrieval Augmented Generation from scratch"} 
        ],
    }
]

response = runnable.invoke(
    {
        "messages": messages,
        "schema": Search.schema(),
        "instructions": instructions,
        "examples": examples,
    }
)
response

{'data': [{'query': 'RAG agent tutorial', 'publish_year': 2023}]}

Trace: https://smith.langchain.com/public/7a95a5af-7f89-4312-90b8-88cc408de1a7/r